In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

In [ ]:
import os

from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.spatial

import anndata
import scanpy as sc
import umap

import torch
from scmg.model.contrastive_embedding import CellEmbedder, embed_adata, decode_cell_state_embedding

from scmg.preprocessing.data_standardization import GeneNameMapper
gene_name_mapper = GeneNameMapper()


In [ ]:
adata = sc.read_h5ad('../ref_cell_adata_measured_count.h5ad')

adata

In [ ]:
all_gene_de_df = pd.read_parquet('cell_type_DE_scores.parquet')
all_gene_de_df

In [ ]:
all_gene_de_df['expressed'] = ((all_gene_de_df['foreground_exp_frac'] > 0.1) 
                             & (all_gene_de_df['foreground_mean'] > 0.1))
all_gene_de_df

In [ ]:
expressed_gene_df = all_gene_de_df[all_gene_de_df['expressed']]
expressed_gene_df

In [ ]:
housekeeping_score_df = adata.var.copy()
housekeeping_score_df['expressed_cell_type_count'] = expressed_gene_df['gene'].value_counts()
housekeeping_score_df['expressed_cell_type_count'].fillna(0, inplace=True)

n_cell_types = len(adata.obs['cell_type'].cat.categories)
housekeeping_score_df['housekeeping_score'] = housekeeping_score_df['expressed_cell_type_count'] / n_cell_types

housekeeping_score_df.to_parquet('housekeeping_score.parquet')
housekeeping_score_df

In [ ]:
housekeeping_score_df['housekeeping_score'].hist(bins=100)